# SteganoGAN in Keras
This notebook contains code attempting to reimplement SteganoGAN in Keras, for the purpose of better understanding (and scrutinizing) it.

*Based on https://github.com/DAI-Lab/SteganoGAN/tree/master/steganogan*

### Modules

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import numpy as np
from keras.losses import BinaryCrossentropy

from models import (
  steganogan_encoder_basic_model,
  steganogan_encoder_residual_model,
  steganogan_encoder_dense_model,

  steganogan_decoder_basic_model,
  steganogan_decoder_dense_model,
  
  steganogan_critic_model
)
from keras_steganogan import KerasSteganoGAN

2024-07-10 21:51:08.286410: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Constants

In [2]:
# Image dimensions
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
IMAGE_CHANNELS = 3

IMAGE_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)
MESSAGE_DEPTH = 2
BATCH_SIZE = 4
MODEL_PATH = 'pretrained_models/steganoGAN_dense.keras'

----

### Call main encode and decode functions (with creating steganographic image and decoding it)

In [4]:
steganoGAN = KerasSteganoGAN(image_shape=IMAGE_SHAPE, data_depth=MESSAGE_DEPTH, model_path=MODEL_PATH)
steganoGAN.encode('input.png', 'images/dense/output.png', 'Hello, World!')
steganoGAN.decode('images/dense/output.png')

'ýÿÿÿÿ^ký\x1a\x00\x02ª\x7fd\x8e\x01\x83õ\x17Xa\x17ÿ¿ßýÿÿù>\x7fÕwýUU]þåíD\x88¹K\x14\x954\t\x11\x83\x15\x18Æ\x00\x04-_WÕWû¿Õ_\x7fß\x80\x05EñA\x11|oy\r÷×¬P+\x80=J\x02ª®ñ}UÀ\x15}Q¾YSQ\x00\x00\x07ÛJ\'3´Á-ZV@`\x80%s«\x03à\x00¡ô\x01\x00\x01]`P%\x97\x1cÜG\x1dZ?\'Õ\x97\x07\x89ÿpªb¬uäÇzªª£ð\x99o\x02X\x9aÞõ\x14ÖÇ}µT\x14¼®ÆÕ\x06×Ó¡Ü\x80¾\x08àjª$.Ö]õQÖÏRßïL^þO?ßw2b\x0e\tf\x96Jd 4(+\x8aj¤/A}µUY¦\x03U\r\x9dé_}W8ósh0\x8dMÚi\x81l4ä_¿ê%&aÝES×Ñ\x04\x95\x19õ\x19\x1díýÁçD\x85Üi&rH\x04ª\x97Ã\x9eÓ[õ´KôOÿÔÈ²÷ \x1d\x03um]\x15tÿ\x95u`°Æ\x88}(1\x01pÿ¼I91EUUUFÝU\x83X-Gõ|´÷×§QäX\x96\x80n$`\x80\x90~X\\1`\x00ÿk¹e#ù\n\x957U5WUWÝÕN\x05\x15°\x80H\x00²§COÕG DïÐ\x1dRà.AtTu\x1d\x15\x15;G÷õQ\x8fýf\x04¡(:".ü?\x1d"]UU`R\x08/h\x8ad×\x94E!ÿÝußßíøD¸8úpÝY\x8acýuÿö?÷Ý\x1dñ\x18\x03U\x15B\x1cUQ]ýVuýýið\x15p\x11qÙvÕUýUAUY~PÙ7¤\x11\x98A\x14W\x16]Á\x95\x91üù\x9b\x83¾\x07\x105\x10\\Q\\¡ý\x17qqRìF\x9c\x84Ý´tÔDL÷M\x10W\x17ÿSD\x86ëè¹\x89»®WAP\x04\x01\x91\x1fD\x0e^âgGQ=@ðmnñE\x1d\x7f\x1dn5]YVýB\x1c\x15=Q=EBYÑ\x1f^P\x1cÍÿUuFu}ÑUÕUmD\x91¹:EÖ

### SteganoGAN predict random data with metrics

In [ ]:
cover_image = tf.random.uniform([1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS], -1, 1, dtype=tf.float32)
message = tf.cast(tf.random.uniform([1, IMAGE_HEIGHT, IMAGE_WIDTH, MESSAGE_DEPTH], 0, 2, dtype=tf.int32), tf.float32)

stego_img, recovered_msg = steganoGAN.predict([cover_image, message])

print("stego_img min: {0}, max: {1}".format(tf.reduce_min(stego_img), tf.reduce_max(stego_img)))
print("recovered_msg min: {0}, max: {1}".format(tf.reduce_min(recovered_msg), tf.reduce_max(recovered_msg)))

print("BinaryCrossentropy: {0}".format(BinaryCrossentropy(from_logits=False)(message, recovered_msg)))
print("PSNR: {0}".format(tf.reduce_mean(tf.image.psnr(cover_image, stego_img, 1))))
print("SSIM: {0}".format(tf.reduce_mean(tf.image.ssim(cover_image, stego_img, 1))))

----

### Testing different functions 

In [ ]:
# Create two random tensors
tensor1 = tf.random.uniform((4, 128, 128, 2), minval=0, maxval=1, dtype=tf.int32)
tensor2 = tf.random.uniform((4, 128, 128, 2), minval=0, maxval=1, dtype=tf.float32)

# Calculate binary crossentropy
loss = BinaryCrossentropy(from_logits=False)
loss = loss(tf.constant([0, 0, 0, 0]), tf.constant([1, 1, 1, 1]))
#loss = loss(tensor1, tensor2)

loss.numpy().astype(np.uint8)

In [ ]:
x = tf.constant([-1, -15, 0, 5])

ones = tf.ones(tf.shape(x))
zeros = tf.zeros(tf.shape(x))

tf.keras.backend.switch(x > 0, ones, zeros)

In [ ]:
x = tf.constant([0, 0, 0, 0])
y = tf.constant([1, 1, 1, 1])

mse = tf.keras.losses.MeanSquaredError() 
mse(x, y).numpy().astype(np.uint8)